In [62]:
import csv
import os
from scipy import stats

In [9]:
#turn weight csv to dict
#method to read in csv and turn contents into a dict
def weightcsvToDict(subreddit):
    
    d = dict()
    
    #path = "weights/" + subreddit + ".csv"
    f = open(subreddit)
    
    for line in f:
        line = line.strip('\n')
        (val, key) = line.split(",")
        d[key] = val
    
    d.pop("feature")
    
    return d

In [22]:
#turn centrality csv to dict
#method to read in csv and turn contents into a dict
def centralitycsvToDict(subreddit):
    
    d = dict()
    
    #path = "weights/" + subreddit + ".csv"
    f = open(subreddit)
    
    for line in f:
        line = line.strip('\n')
        (ind, val, key) = line.split(",")
        d[key] = val
    
    d.pop("1")
    
    return d

In [38]:
# w=weightcsvToDict("../weights/AskMen.csv")
# c=centralitycsvToDict("../centralities/AskMen.csv")

# sw=sorted(w)
# sc=sorted(c)

In [42]:
#for k in sw:
#    print(f"{k}, {w[k]}")

In [43]:
#for k in sc:
#    print(f"{k}, {c[k]}")

In [52]:
#method to sort a dict by key
def sortByKey(d):
    
    result = {}
    
    for k in sorted(d):
        result[k] = d[k]
        
    return result

In [55]:
#list of weight dicts

#get a list of all files in dir
wdirectory = '../weights'
weightsFileList = []
weightsFileNames = []
for filename in os.listdir(wdirectory):
    p = os.path.join(wdirectory, filename)
    weightsFileList.append(p)
    weightsFileNames.append(filename)
    
#put all dicts into a list
allWeightDicts = []
for f in weightsFileList:
    dt = sortByKey(weightcsvToDict(f))
    allWeightDicts.append(dt)

In [56]:
#list of centrality dicts

#get a list of all files in dir
cdirectory = '../centralities'
centralitiesFileList = []
centralitiesFileNames = []
for filename in os.listdir(cdirectory):
    p = os.path.join(cdirectory, filename)
    centralitiesFileList.append(p)
    centralitiesFileNames.append(filename)
    
#put all dicts into a list
allCentralityDicts = []
for f in centralitiesFileList:
    dt = sortByKey(centralitycsvToDict(f))
    allCentralityDicts.append(dt)

In [76]:
# v1=list(allWeightDicts[0].values())
# v2=list(allCentralityDicts[0].values())

In [85]:
# for i in range(len(v1)):
#     v1[i] = float(v1[i])
    
# for i in range(len(v2)):
#     v2[i] = float(v2[i])

In [82]:
#stats.pearsonr(v1, v2)

(0.02257018734556462, 0.08810138412355048)

In [113]:
#method that returns a list of all features
def getIntersection(d1, d2):
    
    d = dict()
    
    s1 = set(d1)
    s2 = set(d2)

    for feature in s1.intersection(s2):
        d[feature] = (d1[feature], d2[feature])
        #print(f"{feature}: {d1[feature]}, {d1[feature]}")
    
    return d

In [133]:
#method that takes in a subreddit and returns corr of weights and centralities
def getCorr(wDict, cDict):
    
    d=getIntersection(wDict, cDict)
    
    v1 = []
    v2 = []
    
    for k in d:
        v1.append(d[k][0])
        v2.append(d[k][1])
    
    #v1=list(wDict.values())
    #v2=list(cDict.values())
    
    #if len(v1) != len(v2):
    #    return
    
    for i in range(len(v1)):
        v1[i] = float(v1[i])
    
    for i in range(len(v2)):
        v2[i] = float(v2[i])
    
    return stats.pearsonr(v1, v2)

In [134]:
getCorr(allWeightDicts[0], allCentralityDicts[0])

(0.022570187345564617, 0.08810138412355048)

In [143]:
finalDict = {}

for i in range(len(allWeightDicts)):
    x=getCorr(allWeightDicts[i], allCentralityDicts[i])
    name = weightsFileNames[i].split('.')[0]
    finalDict[name] = x[0]

In [145]:
{k: v for k, v in sorted(finalDict.items(), key=lambda item: item[1], reverse = True)}

{'TrueUnpopularOpinion': 0.15772605944556695,
 'news': 0.09624514898524408,
 'worldnews': 0.09378449901033208,
 'RedPillWomen': 0.060752929942487974,
 'nba': 0.05977861955542471,
 'MaliciousCompliance': 0.057468478404774044,
 'anime': 0.055539423827778066,
 'Twitch': 0.05294398920310522,
 'Games': 0.05056986548148319,
 'todayilearned': 0.048787180341776946,
 'politics': 0.04684645614524181,
 'personalfinance': 0.04597961174050659,
 'movies': 0.04562069705798781,
 'Conservative': 0.045440828483163984,
 'Minecraft': 0.042802617027364355,
 'nfl': 0.040054262590936085,
 'leagueoflegends': 0.03791202122203246,
 'IAmA': 0.0361539970661069,
 'Cooking': 0.03515498568858452,
 'FemaleDatingStrategy': 0.03314382642958864,
 'Fitness': 0.030753494234032175,
 'Piracy': 0.02989454418661868,
 'MensRights': 0.025971968156630207,
 'relationships': 0.025566413575618473,
 'mildlyinteresting': 0.024890852415230535,
 'AskMen': 0.02345773485967377,
 'tifu': 0.023377038210852373,
 'explainlikeimfive': 0.02257

In [146]:
with open('weightContCorr.csv', 'w') as csvfile:
    
    csvwriter = csv.writer(csvfile)
    
    fields = ['subreddit', 'correlation']
    csvwriter.writerow(fields)

    for k, v in sorted(finalDict.items(), key=lambda item: item[1], reverse = True):
        r = [k, v]
        csvwriter.writerow(r)